In [ ]:
!pip install gradio

In [2]:
import gradio as gr
import torch

In [ ]:
### Update the location of the models below ###
path = '/content/drive/MyDrive/School Work/MEng/ECE1786/Asn2/baseline.pt'
checkpoint1 = torch.load(path)
model = #
model.load_state_dict(checkpoint1)

In [8]:
def getResult(input):
  #############################################
  #pred = model.predict(input)
  pred = '0'
  #############################################

  result = ''
  if(pred == '0'):
    result = "The first sentence is a lie!"
  elif(pred == '1'):
    result = "The second sentence is a lie!"
  elif(pred == '2'):
    result = "The third sentence is a lie!"

  return result

with gr.Blocks() as demo:
  with gr.Column(scale=1, min_width=600):
    text0 = gr.Textbox(label="Sentence 1")
    text1 = gr.Textbox(label="Sentence 2")
    text2 = gr.Textbox(label="Sentence 3")
    predict = gr.Button("Submit")
    text3 = gr.Textbox(label="Prediction")

  predict.click(getResult, inputs=[text0, text1, text2], outputs=text3)

demo.launch(debug=True)

NameError: ignored